In [1]:
import argparse

import os
import sys
import glob
import time
import random

import numpy as np
#from scipy.misc import imread, imresize, imsave
from skimage.io import imread, imsave
from skimage.transform import resize as imresize
from matplotlib import pyplot as plt

from utils.rgb_ind_convertor import *
from utils.util import *

In [2]:
def post_process(input_dir, save_dir, merge=True):
	if not os.path.exists(save_dir):
		os.mkdir(save_dir)

	input_paths = sorted(glob.glob(os.path.join(input_dir, '*.png')))
	names = [i.split('/')[-1] for i in input_paths]
	out_paths = [os.path.join(save_dir, i) for i in names]

	n = len(input_paths)
	# n = 1
	for i in range(n):
		im = imread(input_paths[i], pilmode='RGB')
		im_ind = rgb2ind(im, color_map=floorplan_fuse_map)
		# seperate image into room-seg & boundary-seg
		rm_ind = im_ind.copy()
		rm_ind[im_ind==9] = 0
		rm_ind[im_ind==10] = 0

		bd_ind = np.zeros(im_ind.shape, dtype=np.uint8)
		bd_ind[im_ind==9] = 9
		bd_ind[im_ind==10] = 10

		hard_c = (bd_ind>0).astype(np.uint8)

		# region from room prediction it self
		rm_mask = np.zeros(rm_ind.shape)
		rm_mask[rm_ind>0] = 1			
		# region from close_wall line		
		cw_mask = hard_c
		# refine close wall mask by filling the grap between bright line	
		cw_mask = fill_break_line(cw_mask)
			
		fuse_mask = cw_mask + rm_mask
		fuse_mask[fuse_mask>=1] = 255

		# refine fuse mask by filling the hole
		fuse_mask = flood_fill(fuse_mask)
		fuse_mask = fuse_mask // 255	

		# one room one label
		new_rm_ind = refine_room_region(cw_mask, rm_ind)

		# ignore the background mislabeling
		new_rm_ind = fuse_mask*new_rm_ind

		print('Saving {}th refined room prediciton to {}'.format(i, out_paths[i]))
		if merge:
			new_rm_ind[bd_ind==9] = 9
			new_rm_ind[bd_ind==10] = 10
			rgb = ind2rgb(new_rm_ind, color_map=floorplan_fuse_map)
		else:
			rgb = ind2rgb(new_rm_ind)
		imsave(out_paths[i], rgb)



In [3]:
input_dir = '/d2/studies/TF2DeepFloorplan/dataset/newyork/train/'
save_dir = '/d2/studies/TF2DeepFloorplan/dataset/results/'

In [4]:
post_process(input_dir, save_dir, merge=True)

/d2/studies/TF2DeepFloorplan/utils/util.py:26: UserWarning: ignoring size because footprint is set
  output_array = np.maximum(input_array,ndimage.grey_erosion(output_array, size=(3,3), footprint=el))
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


Saving 0th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/10_close.png


Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


Saving 1th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/10_close_wall.png


Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


...

Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


Saving 19th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/2_wall.png


Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


Saving 20th refined room prediciton to /d2/studies/TF2DeepFloorplan/dataset/results/30044076_close.png


KeyboardInterrupt: 